In [1]:
from symred.chem import xyz_from_pubchem
import numpy as np
from openfermion import FermionOperator
from scipy.sparse.linalg import expm
from openfermion import get_sparse_operator

In [ ]:
# mol_name = 'H2O'
# mol_name = 'H2'

# mol_name = 'LiH'

In [ ]:
xyz_file = xyz_from_pubchem(mol_name)
print(xyz_file)

In [2]:
# xyz_file = '4\n \nH\t0\t0\t0\nH\t1\t0\t0\nH\t2\t0\t0\nH\t3\t0\t0'

In [3]:
from symred.chem import Draw_molecule

viewer = Draw_molecule(xyz_file,
                       width=400,
                       height=400,
                       style="sphere")
viewer.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [4]:
from symred.chem import PySCFDriver

In [5]:
basis = 'STO-3G'
convergence = 1e6
charge=0
max_hf_cycles=50
ram = 8_000
run_mp2  = True,
run_cisd = True,
run_ccsd = True,
run_fci  = True

In [6]:
pyscf_obj = PySCFDriver(xyz_file,
                       basis,
                       convergence=convergence,
                       charge=charge,
                       max_ram_memory=ram,
                       max_hf_cycles=max_hf_cycles,
                       
                       run_mp2=run_mp2,
                       run_cisd=run_cisd,
                       run_ccsd=run_ccsd,
                       run_fci=run_fci)

In [7]:
pyscf_obj.run_pyscf()

In [8]:
2*pyscf_obj.pyscf_hf.mol.nao

8

In [9]:
from symred.chem import FermionicHamilt

In [10]:
H_ferm = FermionicHamilt(pyscf_obj.pyscf_hf)

H_ferm.build_operator()

H_ferm.fermionic_molecular_hamiltonian

() 2.29310124732
((0, 1), (0, 0)) -1.8385126260957712
((0, 1), (4, 0)) 0.1535200531800066
((1, 1), (1, 0)) -1.8385126260957712
((1, 1), (5, 0)) 0.1535200531800066
((2, 1), (2, 0)) -1.5485198738611377
((2, 1), (6, 0)) 0.1346528866850458
((3, 1), (3, 0)) -1.5485198738611377
((3, 1), (7, 0)) 0.1346528866850458
((4, 1), (0, 0)) 0.15352005318000683
((4, 1), (4, 0)) -1.2423978239696827
((5, 1), (1, 0)) 0.15352005318000683
((5, 1), (5, 0)) -1.2423978239696827
((6, 1), (2, 0)) 0.13465288668504574
((6, 1), (6, 0)) -0.9084576464931442
((7, 1), (3, 0)) 0.13465288668504574
((7, 1), (7, 0)) -0.9084576464931442
((0, 1), (0, 1), (0, 0), (0, 0)) 0.2504327702741458
((0, 1), (0, 1), (0, 0), (4, 0)) -0.04165187456702536
((0, 1), (0, 1), (2, 0), (2, 0)) 0.07725837026056045
((0, 1), (0, 1), (2, 0), (6, 0)) -0.02286714657234941
((0, 1), (0, 1), (4, 0), (0, 0)) -0.041651874567025335
((0, 1), (0, 1), (4, 0), (4, 0)) 0.053084040051590224
((0, 1), (0, 1), (6, 0), (2, 0)) -0.022867146572349395
((0, 1), (0, 1), (

In [11]:
hf_state = H_ferm.hf_comp_basis_state
hf_state


array([1, 1, 1, 1, 0, 0, 0, 0])

In [12]:
hf_ket = H_ferm.hf_ket
hf_ket.shape

(256,)

In [13]:
## check
from functools import reduce
state = {0: np.array([[1],[0]]), 1: np.array([[0],[1]])}
ll = [state[bit] for bit in hf_state]
kket = reduce(np.kron, ll)
all(kket[:,0] == hf_ket)

True

In [14]:
print(H_ferm.scf_method.energy_tot())
print(pyscf_obj.pyscf_hf.energy_tot())

H_mat = H_ferm.get_sparse_ham()
hf_ket.conj().T @ H_mat @ hf_ket

-2.0982818585784786
-2.0982818585784786


(-2.0982818585784786+0j)

In [ ]:
# def compute_mp2_amplitudes(self) -> ClosedShellAmplitudes:
#     """
#     Compute closed-shell mp2 amplitudes
#     .. math::
#         t(a,i,b,j) = 0.25 * g(a,i,b,j)/(e(i) + e(j) -a(i) - b(j) )
#     :return:
#     Parameters
#     ----------
#     Returns
#     -------
#     """
#     g = self.molecule.two_body_integrals
#     fij = self.molecule.orbital_energies
#     nocc = self.molecule.n_electrons // 2  # this is never the active space
#     ei = fij[:nocc]
#     ai = fij[nocc:]
#     abgij = g[nocc:, nocc:, :nocc, :nocc]
#     amplitudes = abgij * 1.0 / (
#             ei.reshape(1, 1, -1, 1) + ei.reshape(1, 1, 1, -1) - ai.reshape(-1, 1, 1, 1) - ai.reshape(1, -1, 1, 1))
#     E = 2.0 * numpy.einsum('abij,abij->', amplitudes, abgij) - numpy.einsum('abji,abij', amplitudes, abgij,
#                                                                             optimize='greedy')

#     self.molecule.mp2_energy = E + self.molecule.hf_energy
#     return ClosedShellAmplitudes(tIjAb=numpy.einsum('abij -> ijab', amplitudes, optimize='greedy'))

In [15]:
one_body_integrals = H_ferm._one_body_integrals
two_body_integrals = H_ferm._two_body_integrals

g = two_body_integrals
fij = pyscf_obj.pyscf_hf.mo_energy

nocc = H_ferm.n_electrons // 2  # this is never the active space
ei = fij[:nocc]
ai = fij[nocc:]
abgij = g[nocc:, nocc:, :nocc, :nocc]
amplitudes = abgij * 1.0 / (
        ei.reshape(1, 1, -1, 1) + ei.reshape(1, 1, 1, -1) - ai.reshape(-1, 1, 1, 1) - ai.reshape(1, -1, 1, 1))
E = 2.0 * np.einsum('abij,abij->', amplitudes, abgij) - np.einsum('abji,abij', amplitudes, abgij,
                                                                        optimize='greedy')

E

-0.04160571461928189

In [16]:
pyscf_obj.pyscf_mp2.e_corr

-0.04160571461928188

In [ ]:
tIjAb=np.einsum('abij -> ijab', amplitudes, optimize='greedy')

tIjAb.shape

In [ ]:
# tIjAb=np.einsum('abij -> abij', amplitudes, optimize='greedy')

# tIjAb.shape

In [ ]:
# pyscf_obj.pyscf_mp2.t2 # <-- these are the amplitudes!

In [ ]:
# variables = {}
# if tIjAb is not None:
#     nvirt = tIjAb.shape[2]
#     nocc = tIjAb.shape[0]
#     assert (tIjAb.shape[1] == nocc and tIjAb.shape[3] == nvirt)
#     for (I, J, A, B), value in np.ndenumerate(tIjAb):
#         if not np.isclose(value, 0.0, atol=1e-8):
#             variables[(nocc + A, I, nocc + B, J)] = value
            

nvirt = tIjAb.shape[2]
nocc = tIjAb.shape[0]
excite_op =FermionOperator()
assert (tIjAb.shape[1] == nocc and tIjAb.shape[3] == nvirt)
for (I, J, A, B), value in np.ndenumerate(tIjAb):
    if not np.isclose(value, 0.0, atol=1e-8):
        excite_op+= FermionOperator(f'{nocc + A}^ {I}^ {nocc + B} {J}', value)
#         excite_op+= FermionOperator(f'{I}^ {J}^ {nocc + A} {nocc + B}', value)
#         excite_op+= FermionOperator(f'{nocc + A}^ {nocc + B}^ {I} {J}', -value)

In [ ]:
tijab = tIjAb - np.einsum("ijab -> ijba", tIjAb, optimize='greedy')

nvirt = tijab.shape[2]
nocc = tijab.shape[0]
excite_op =FermionOperator()
assert (tijab.shape[1] == nocc and tijab.shape[3] == nvirt)
for (I, J, A, B), value in np.ndenumerate(tijab):
    if not np.isclose(value, 0.0, atol=1e-8):
        excite_op+= FermionOperator(f'{nocc + A}^ {I}^ {nocc + B} {J}', value)

In [ ]:
tijab = tIjAb - np.einsum("ijab -> ijba", tIjAb, optimize='greedy')

tIjAb=tIjAb
tiJaB=tIjAb
tijab=tijab
tIJAB=tijab


threshold = 1e-8
excite_op =FermionOperator()
nvirt = tIjAb.shape[2]
nocc = tIjAb.shape[0]
assert (tIjAb.shape[1] == nocc and tIjAb.shape[3] == nvirt)

for (I, j, A, b), value in np.ndenumerate(tIjAb):
    if not np.isclose(value, 0.0, atol=threshold):
        excite_op+= FermionOperator(f'{2 * (nocc + A)}^ {2*I}^ {2 * (nocc + b) + 1} {j + 1}', value/2)
#         variables[(2 * (nocc + A), 2 * I, 2 * (nocc + b) + 1, j + 1)] = value
for (i, J, a, B), value in np.ndenumerate(tiJaB):
    if not np.isclose(value, 0.0, atol=threshold):
        excite_op+= FermionOperator(f'{2 * (nocc + a)+1}^ {2*i+1}^ {2 * (nocc + B)} {J}', value/2)
#         variables[(2 * (nocc + a) + 1, 2 * i + 1, 2 * (nocc + B), J)] = value
for (i, j, a, b), value in np.ndenumerate(tijab):
    if not np.isclose(value, 0.0, atol=threshold):
        excite_op+= FermionOperator(f'{2 * (nocc + a)+1}^ {2*i+1}^ {2 * (nocc + b)+1} {j+1}', value/2)
#         variables[(2 * (nocc + a) + 1, 2 * i + 1, 2 * (nocc + b) + 1, j + 1)] = value
for (I, J, A, B), value in np.ndenumerate(tijab):
    if not np.isclose(value, 0.0, atol=threshold):
#         variables[(2 * (nocc + A), 2 * I, 2 * (nocc + B), J)] = value
        excite_op+= FermionOperator(f'{2 * (nocc + A)}^ {2*I}^ {2 * (nocc + B)} {J}', value/2)

In [ ]:
# tijab = tIjAb - np.einsum("ijab -> ijba", tIjAb, optimize='greedy')

# tIjAb=tIjAb
# tiJaB=tIjAb
# tijab=tijab
# tIJAB=tijab


# threshold = 1e-8
# excite_op =FermionOperator()
# nvirt = tIjAb.shape[2]
# nocc = tIjAb.shape[0]
# assert (tIjAb.shape[1] == nocc and tIjAb.shape[3] == nvirt)

# for (I, j, A, b), value in np.ndenumerate(tIjAb):
#     if not np.isclose(value, 0.0, atol=threshold):
#         term = [2 * (nocc + A), 2*I, 2 * (nocc + b) + 1, j + 1]
#         excite_op+= FermionOperator(f'{term[0]}^ {term[1]}^ {term[2]} {term[3]}', value/2)
#         excite_op+= FermionOperator(f'{2*term[0]+1}^ {2*term[1]+1}^ {2*term[2]+1} {2*term[3]+1}', value/2)
# #         variables[(2 * (nocc + A), 2 * I, 2 * (nocc + b) + 1, j + 1)] = value
# for (i, J, a, B), value in np.ndenumerate(tiJaB):
#     if not np.isclose(value, 0.0, atol=threshold):
#         term = [2 * (nocc + a)+1, 2*i+1, 2 * (nocc + B), J]
#         excite_op+= FermionOperator(f'{term[0]}^ {term[1]}^ {term[2]} {term[3]}', value/2)
#         excite_op+= FermionOperator(f'{2*term[0]+1}^ {2*term[1]+1}^ {2*term[2]+1} {2*term[3]+1}', value/2)
# #         variables[(2 * (nocc + a) + 1, 2 * i + 1, 2 * (nocc + B), J)] = value
# for (i, j, a, b), value in np.ndenumerate(tijab):
#     if not np.isclose(value, 0.0, atol=threshold):
#         term = [2 * (nocc + a)+1, 2*i+1, 2 * (nocc + b)+1, j+1]
#         excite_op+= FermionOperator(f'{term[0]}^ {term[1]}^ {term[2]} {term[3]}', value/2)
#         excite_op+= FermionOperator(f'{2*term[0]+1}^ {2*term[1]+1}^ {2*term[2]+1} {2*term[3]+1}', value/2)
# #         variables[(2 * (nocc + a) + 1, 2 * i + 1, 2 * (nocc + b) + 1, j + 1)] = value
# for (I, J, A, B), value in np.ndenumerate(tijab):
#     if not np.isclose(value, 0.0, atol=threshold):
#         term = [2 * (nocc + A), 2*I, 2 * (nocc + B), J]
#         excite_op+= FermionOperator(f'{term[0]}^ {term[1]}^ {term[2]} {term[3]}', value/2)
#         excite_op+= FermionOperator(f'{2*term[0]+1}^ {2*term[1]+1}^ {2*term[2]+1} {2*term[3]+1}', value/2)

In [ ]:
len(list(excite_op))

# excite_op/2

In [ ]:
len(list(generator))

In [ ]:
generator

In [ ]:
mp2_ansatz = expm(get_sparse_operator(0.25*excite_op, n_qubits=H_ferm.n_qubits))
## mp2_ansatz = expm(get_sparse_operator(excite_op, n_qubits=H_ferm.n_qubits))

## help(qubit_operator_sparse)

In [ ]:
mp2_ket =  mp2_ansatz @ hf_ket
mp2_ket.conj().T @ H_mat @ mp2_ket

In [ ]:
pyscf_obj.pyscf_mp2.e_tot

In [ ]:
pyscf_obj.pyscf_mp2.e_hf

In [ ]:
hf_ket.conj().T @ mp2_ket

In [ ]:
# t(a,i,b,j) = 0.25 * g(a,i,b,j)/(e(i) + e(j) -a(i) - b(j) )

# +aibj
# -abij

In [ ]:
pyscf_obj.pyscf_mp2.e_corr #+ pyscf_obj.pyscf_mp2.e_corr

In [ ]:
variables = {}
if tIjAb is not None:
    nvirt = tIjAb.shape[2]
    nocc = tIjAb.shape[0]
    assert (tIjAb.shape[1] == nocc and tIjAb.shape[3] == nvirt)
    for (I, J, A, B), value in np.ndenumerate(tIjAb):
        if not np.isclose(value, 0.0, atol=1e-8):
            variables[(nocc + A, I, nocc + B, J)] = value

dict(sorted(variables.items(), key=lambda x: np.abs(x[1]), reverse=True))

In [17]:
from pyscf.cc.addons import spatial2spin

t2 = spatial2spin(pyscf_obj.pyscf_mp2.t2)
no, nv = t2.shape[1:3]
nmo = no + nv
double_amps = np.zeros((nmo, nmo, nmo, nmo))
double_amps[no:,:no,no:,:no] = .5 * t2.transpose(2,0,3,1)

In [18]:
# double_amplitudes_list=[]
# double_amplitudes = double_amps
# for i, j, k, l in zip(*double_amplitudes.nonzero()):
#     double_amplitudes_list.append([[i, j, k, l],
#                                    double_amplitudes[i, j, k, l]])
    
# double_amplitudes_list

double_amplitudes_list=[]
double_amplitudes = double_amps
for i, j, k, l in zip(*double_amplitudes.nonzero()):
    if not np.isclose(double_amplitudes[i, j, k, l], 0):
        double_amplitudes_list.append([[i, j, k, l],
                                       double_amplitudes[i, j, k, l]])
    
double_amplitudes_list

[[[4, 0, 5, 1], -0.028914558866912243],
 [[4, 0, 6, 2], -0.010778158725408358],
 [[4, 0, 7, 3], -0.022405585894956476],
 [[4, 1, 5, 0], 0.028914558866912243],
 [[4, 1, 7, 2], 0.011627427169548121],
 [[4, 2, 5, 3], -0.05280759309647757],
 [[4, 2, 6, 0], 0.010778158725408347],
 [[4, 2, 7, 1], -0.011627427169548121],
 [[4, 3, 5, 2], 0.05280759309647757],
 [[4, 3, 7, 0], 0.022405585894956476],
 [[5, 0, 4, 1], 0.028914558866912243],
 [[5, 0, 6, 3], 0.011627427169548118],
 [[5, 1, 4, 0], -0.028914558866912243],
 [[5, 1, 6, 2], -0.02240558589495647],
 [[5, 1, 7, 3], -0.010778158725408358],
 [[5, 2, 4, 3], 0.05280759309647757],
 [[5, 2, 6, 1], 0.02240558589495647],
 [[5, 3, 4, 2], -0.05280759309647757],
 [[5, 3, 6, 0], -0.011627427169548118],
 [[5, 3, 7, 1], 0.010778158725408347],
 [[6, 0, 4, 2], 0.010778158725408358],
 [[6, 0, 5, 3], -0.011627427169548118],
 [[6, 0, 7, 1], -0.016527871022874608],
 [[6, 1, 5, 2], 0.02240558589495647],
 [[6, 1, 7, 0], 0.016527871022874608],
 [[6, 2, 4, 0], -0.0

In [19]:
generator = FermionOperator()

# Add double excitations
for (i, j, k, l), t_ijkl in double_amplitudes_list:
    i, j, k, l = int(i), int(j), int(k), int(l)
    generator += FermionOperator(((i, 1), (j, 0), (k, 1), (l, 0)), t_ijkl)
#     if anti_hermitian:
#         generator += FermionOperator(((l, 1), (k, 0), (j, 1), (i, 0)),
#                                      -t_ijkl)

In [20]:
generator

-0.028914558866912243 [4^ 0 5^ 1] +
-0.010778158725408358 [4^ 0 6^ 2] +
-0.022405585894956476 [4^ 0 7^ 3] +
0.028914558866912243 [4^ 1 5^ 0] +
0.011627427169548121 [4^ 1 7^ 2] +
-0.05280759309647757 [4^ 2 5^ 3] +
0.010778158725408347 [4^ 2 6^ 0] +
-0.011627427169548121 [4^ 2 7^ 1] +
0.05280759309647757 [4^ 3 5^ 2] +
0.022405585894956476 [4^ 3 7^ 0] +
0.028914558866912243 [5^ 0 4^ 1] +
0.011627427169548118 [5^ 0 6^ 3] +
-0.028914558866912243 [5^ 1 4^ 0] +
-0.02240558589495647 [5^ 1 6^ 2] +
-0.010778158725408358 [5^ 1 7^ 3] +
0.05280759309647757 [5^ 2 4^ 3] +
0.02240558589495647 [5^ 2 6^ 1] +
-0.05280759309647757 [5^ 3 4^ 2] +
-0.011627427169548118 [5^ 3 6^ 0] +
0.010778158725408347 [5^ 3 7^ 1] +
0.010778158725408358 [6^ 0 4^ 2] +
-0.011627427169548118 [6^ 0 5^ 3] +
-0.016527871022874608 [6^ 0 7^ 1] +
0.02240558589495647 [6^ 1 5^ 2] +
0.016527871022874608 [6^ 1 7^ 0] +
-0.010778158725408347 [6^ 2 4^ 0] +
-0.02240558589495647 [6^ 2 5^ 1] +
-0.020755795986627187 [6^ 2 7^ 3] +
0.01162742716

In [21]:
mp2_ansatz = expm(get_sparse_operator(0.25*generator, n_qubits=H_ferm.n_qubits))
# mp2_ansatz = expm(get_sparse_operator(0.125*generator, n_qubits=H_ferm.n_qubits))

In [22]:
mp2_ket =  mp2_ansatz @ hf_ket
mp2_ket.conj().T @ H_mat @ mp2_ket

(-2.145791117472204+0j)

In [23]:
pyscf_obj.pyscf_mp2.e_tot

-2.1398875731977607

In [24]:
one_body_integrals = H_ferm._one_body_integrals
two_body_integrals = H_ferm._two_body_integrals

g = two_body_integrals
fij = pyscf_obj.pyscf_hf.mo_energy

nocc = H_ferm.n_electrons // 2  # this is never the active space
ei = fij[:nocc]
ai = fij[nocc:]
V = g[nocc:, nocc:, :nocc, :nocc]
amplitudes = abgij * 1.0 / (
        ei.reshape(1, 1, -1, 1) + ei.reshape(1, 1, 1, -1) - ai.reshape(-1, 1, 1, 1) - ai.reshape(1, -1, 1, 1))
E = 2.0 * np.einsum('abij,abij->', amplitudes, abgij) - np.einsum('abji,abij', amplitudes, abgij,
                                                                        optimize='greedy')

E

-0.04160571461928189

https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.154.2423&rep=rep1&type=pdf

In [25]:
mp2_ansatz = expm(get_sparse_operator(0.5*generator, n_qubits=H_ferm.n_qubits))
# mp2_ansatz = expm(get_sparse_operator(0.125*generator, n_qubits=H_ferm.n_qubits))

In [26]:
mp2_ket =  mp2_ansatz @ hf_ket
hf_ket.conj().T @ H_mat @ mp2_ket

(-2.1398875731977607+0j)

In [27]:
pyscf_obj.pyscf_mp2.e_corr

-0.04160571461928188

In [28]:
(hf_ket.conj().T @ H_mat @ mp2_ket - pyscf_obj.pyscf_mp2.e_hf).real

-0.041605714619282086